In [1]:
dir_train_anon = '../Data/raw/echr/EN_train_Anon'
dir_train = '../Data/raw/echr/EN_train'
dir_test_anon = '../Data/raw/echr/EN_test_Anon'
dir_test = '../Data/raw/echr/EN_test'

In [2]:
import os
import sys

def list_all_files(directory):
    """
    列出目录下的所有文件（不包括子目录）
    :param directory: 目标目录路径
    :return: 文件路径列表
    """
    files = []
    for item in os.listdir(directory):
        full_path = os.path.join(directory, item)
        if os.path.isfile(full_path):  # 判断是否为文件
            files.append(full_path)
    return files

paths_train_anon = list_all_files(dir_train_anon)
paths_train = list_all_files(dir_train)
paths_test_anon = list_all_files(dir_test_anon)
paths_test = list_all_files(dir_test)

In [3]:
import json
from datasets import Dataset

def read_json_file(file_path):
    """
    读取 JSON 文件并返回解析后的数据
    :param file_path: JSON 文件路径
    :return: 解析后的 Python 数据结构（通常是字典或列表）
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)  # 将 JSON 转换为 Python 对象
    return data

# data_train_anon = [j for i in paths_train_anon for j in read_json_file(i)['TEXT']]
data_train = [j for i in paths_train for j in read_json_file(i)['TEXT']]
data_test = [j for i in paths_test for j in read_json_file(i)['TEXT']]

dataset_train = Dataset.from_dict({'text':data_train})
dataset_test = Dataset.from_dict({'text':data_test})

In [4]:
dataset_train

Dataset({
    features: ['text'],
    num_rows: 303483
})

In [5]:
dataset_test

Dataset({
    features: ['text'],
    num_rows: 94150
})

### 处理数据


In [6]:
import os
os.environ['https_proxy'] = 'http://10.14.30.39:7890'

from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine, DeanonymizeEngine
from presidio_analyzer.nlp_engine import TransformersNlpEngine
from presidio_anonymizer.entities import RecognizerResult, OperatorResult, OperatorConfig
from presidio_anonymizer.operators import Decrypt
from presidio_anonymizer.entities import (
    ConflictResolutionStrategy,
    EngineResult,
    OperatorConfig,
    RecognizerResult,
)
from presidio_analyzer.nlp_engine import NlpEngineProvider

import sys
sys.path.append('..')
from Data.utils import save_json

import copy

configuration = {
    "nlp_engine_name": "spacy",
    "models": [{"lang_code": "es", "model_name": "es_core_news_md"},
                {"lang_code": "en", "model_name": "en_core_web_lg"}],
}

provider = NlpEngineProvider(nlp_configuration=configuration)
nlp_engine_with_spanish = provider.create_engine()

analyzer = AnalyzerEngine(default_score_threshold=0.85,nlp_engine=nlp_engine_with_spanish, 
    supported_languages=["en"])
anonymizer = AnonymizerEngine()

def presidio_pii(seq,analyzer,anonymizer):
    # print(example)
    # ,entities=['PERSON', 'PHONE_NUMBER', 'DATE_TIME', 'LOCATION', 'EMAIL_ADDRESS', 'NRP']÷
    analyze_result = analyzer.analyze(text=seq,language='en',entities=['PERSON', 'DATE_TIME', 'LOCATION', 'NRP'])
    # analyze_result = analyzer.analyze(text=seq,language='en')

    op_dict = {}
    pii_mask_idx = []
    pii_mask = []
    pii_dict = {} # pii_type:[pii1,pii2]
    masked_seq = copy.deepcopy(seq)
    # print(analyze_result)
    for az in analyze_result:
        pii = seq[az.start:az.end]
        pii_type = az.entity_type
        if pii_type not in pii_dict.keys():
            pii_dict[pii_type] = [pii]
        else:
            if pii not in pii_dict[pii_type]:
                pii_dict[pii_type].append(pii)
        idx = pii_dict[pii_type].index(pii)
        pii_type_new = pii_type+'-'+str(idx)
        masked_seq = masked_seq.replace(pii,f'[{pii_type_new}]')
        
        pii_mask_idx.append({'value':pii,'label':pii_type_new,'start':az.start,'end':az.end})
        pii_mask.append((pii,pii_type_new))

    pii_mask = set(pii_mask)
    pii_mask = [{'value':pm[0],'label':pm[1]} for pm in pii_mask]
    return masked_seq,pii_mask,pii_mask_idx

def proc_pii(example):
    body = example['text']
    data = {}
    data['unmask_seq'] = body
    masked_seq,pii_mask,pii_mask_idx = presidio_pii(body,analyzer,anonymizer)
    # print(masked_seq)
    data['masked_seq'] = masked_seq
    data['pii_mask_idx'] = pii_mask_idx
    data['pii_mask'] = pii_mask
    return data

def filter_conflict(example):
    pii_mask = example['pii_mask']
    flag = 0
    pii_dict = {}
    for pm in pii_mask:
        pii_type = pm['label']
        pii = pm['value']
        if pii_type not in pii_dict:
            pii_dict[pii_type] = pii
        else:
            if pii_dict[pii_type] != pii:
                flag=1
    return flag==0


def fliter_pii_too_much(example):
    # 句子中90%单词不能是pii
    len_unmask_seq = len(example['masked_seq'].split(' '))
    len_pii_mask = len(example['pii_mask_idx'])
    # 一句话中一种PII不应该超过三种以上
    return len_pii_mask <= 0.1*len_unmask_seq

def filter_pii_type_too_much(example):
    flag = 0
    pii_mask=example['pii_mask']
    # print(set(pii_mask))
    for pm in pii_mask:
        label = pm['label']
        if '4' in label:
            flag=1
    return flag==0 and len(pii_mask)<7

# 字符串长度超过500的
dataset_train_proc = dataset_train.filter(lambda x: len(x['text'])<300,num_proc=48)
print(dataset_train_proc)
# 字符串长度小于100的
dataset_train_proc = dataset_train_proc.filter(lambda x: len(x['text'])>200,num_proc=48)
print(dataset_train_proc)

dataset_train_proc = dataset_train_proc.map(proc_pii,num_proc=48)
print(dataset_train_proc)

# 过滤没有pii的数据
dataset_train_proc = dataset_train_proc.filter(lambda x: len(x['pii_mask'])>0, num_proc=48)
print(dataset_train_proc)

# 过滤句子中90%单词不能是pii
dataset_train_proc = dataset_train_proc.filter(fliter_pii_too_much,num_proc=48)
print(dataset_train_proc)

# 过滤句子中pii类型过多
dataset_train_proc = dataset_train_proc.filter(filter_pii_type_too_much,num_proc=48)
print(dataset_train_proc)

Filter (num_proc=48):   0%|          | 0/303483 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 179201
})


Filter (num_proc=48):   0%|          | 0/179201 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 55180
})


Map (num_proc=48):   0%|          | 0/55180 [00:00<?, ? examples/s]

...

Dataset({
    features: ['text', 'unmask_seq', 'masked_seq', 'pii_mask_idx', 'pii_mask'],
    num_rows: 38823
})


Filter (num_proc=48):   0%|          | 0/38823 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'unmask_seq', 'masked_seq', 'pii_mask_idx', 'pii_mask'],
    num_rows: 33056
})


Filter (num_proc=48):   0%|          | 0/33056 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'unmask_seq', 'masked_seq', 'pii_mask_idx', 'pii_mask'],
    num_rows: 33056
})


In [7]:
import sys
sys.path.append('..')
from Data.utils import save_json
dataset_AB = dataset_train_proc.train_test_split(test_size=0.5,seed=42)

path = '../Data/raw/phishing/echr.json'

save_json({'A':dataset_AB['train'].to_list(),'B':dataset_AB['test'].to_list()},path)

In [8]:
pii_dict = {}
for pm in dataset_train_proc['pii_mask']:
    for p in pm:
        label = p['label'].split("-")[0]
        value = p['value']
        if label not in pii_dict.keys():
            pii_dict[label] = 1
        else:
            pii_dict[label] +=1
pii_dict

{'LOCATION': 7106, 'DATE_TIME': 41710, 'NRP': 2502, 'PERSON': 9087}

### 验证的数据

In [22]:
len_mate_data = [len(i) for i in dataset_train['text']]


import pandas as pd

def calculate_quartiles_with_pandas(data):
    """
    使用 Pandas 计算数据的四分位数
    :param data: 数据列表或数组
    :return: 第 1 四分位数、第 2 四分位数（中位数）、第 3 四分位数
    """
    series = pd.Series(data)
    q1 = series.quantile(0.25)  # 第 1 四分位数
    q2 = series.quantile(0.50)  # 中位数
    q3 = series.quantile(0.75)  # 第 3 四分位数
    return q1, q2, q3

# 示例用法
data = len_mate_data
q1, q2, q3 = calculate_quartiles_with_pandas(data)
print(f"第 1 四分位数: {q1}, 中位数: {q2}, 第 3 四分位数: {q3}")
print('mean:',sum(len_mate_data)/len(len_mate_data))

第 1 四分位数: 131.0, 中位数: 246.0, 第 3 四分位数: 431.0
mean: 326.0670185809419


In [ ]:
import pandas as pd

def calculate_quartiles_with_pandas(data):
    """
    使用 Pandas 计算数据的四分位数
    :param data: 数据列表或数组
    :return: 第 1 四分位数、第 2 四分位数（中位数）、第 3 四分位数
    """
    series = pd.Series(data)
    q1 = series.quantile(0.25)  # 第 1 四分位数
    q2 = series.quantile(0.50)  # 中位数
    q3 = series.quantile(0.75)  # 第 3 四分位数
    return q1, q2, q3

# 示例用法
data = [7, 15, 36, 39, 40, 41, 42, 43, 46, 49]
q1, q2, q3 = calculate_quartiles_with_pandas(data)
print(f"第 1 四分位数: {q1}, 中位数: {q2}, 第 3 四分位数: {q3}")

In [19]:
pii_dict = {}
for pm in dataset_train_proc['pii_mask']:
    for p in pm:
        label = p['label'].split('-')[0]
        value = p['value']
        if label not in pii_dict.keys():
            pii_dict[label] = 1
        else:
            pii_dict[label] +=1
pii_dict

{'LOCATION': 52447,
 'DATE_TIME': 254849,
 'NRP': 21073,
 'PERSON': 70134,
 'URL': 20,
 'AU_ACN': 9,
 'AU_TFN': 11,
 'MEDICAL_LICENSE': 1,
 'UK_NHS': 4}

In [11]:
dataset_train_proc['pii_mask'][:20]

[[{'label': 'LOCATION-0', 'value': 'the United Kingdom'},
  {'label': 'DATE_TIME-1', 'value': 'fifteen-year'},
  {'label': 'DATE_TIME-0', 'value': 'less than fifteen years'},
  {'label': 'NRP-0', 'value': 'British'},
  {'label': 'DATE_TIME-2', 'value': '5 May 2010'}],
 [{'label': 'DATE_TIME-0', 'value': '1983'},
  {'label': 'LOCATION-0', 'value': 'the Republic of Ireland'}],
 [{'label': 'DATE_TIME-0', 'value': '1983'},
  {'label': 'LOCATION-0', 'value': 'the United Kingdom'}],
 [{'label': 'LOCATION-0', 'value': 'the United Kingdom'}],
 [{'label': 'DATE_TIME-0', 'value': '1985'},
  {'label': 'NRP-0', 'value': 'British'},
  {'label': 'LOCATION-0', 'value': 'the United Kingdom'}],
 [{'label': 'DATE_TIME-0', 'value': '15 years ending'}],
 [{'label': 'NRP-0', 'value': 'British'},
  {'label': 'LOCATION-0', 'value': 'the United Kingdom'}],
 [{'label': 'DATE_TIME-0', 'value': '2(4'}],
 [{'label': 'DATE_TIME-0', 'value': '1985'},
  {'label': 'NRP-0', 'value': 'British'},
  {'label': 'LOCATION-0

In [1]:
def find_differences(str1, str2):
    """
    查找两个字符串中的不同子串并以字典形式返回。
    返回格式: {'str1_diff': [...], 'str2_diff': [...]}
    """
    import difflib

    # 使用 difflib.SequenceMatcher 查找不同部分
    matcher = difflib.SequenceMatcher(None, str1, str2)
    str1_diff = []
    str2_diff = []

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == 'replace' or tag == 'delete':
            str1_diff.append(str1[i1:i2])
        if tag == 'replace' or tag == 'insert':
            str2_diff.append(str2[j1:j2])

    return {'str1_diff': str1_diff, 'str2_diff': str2_diff}


# 示例字符串
str1 = "abcdefg"
str2 = "abcxyfg"

# 查找不同部分
differences = find_differences(str1, str2)

# 打印结果
print("不一样的子串: ", differences)

不一样的子串:  {'str1_diff': ['de'], 'str2_diff': ['xy']}


In [2]:
str1="The applicant, born in 1968, presently resides in Karlskrona, Sweden. In his application form, he states that he has Croatian nationality.", "", "On 23 March 1994 the applicant arrived in Sweden and requested asylum. He stated that he was a Bosnian Catholic with Bosnian citizenship. Before leaving for Sweden, he had been living in Kraljeva Sutjeska. He held also a Croatian passport but this was to be considered as a travel document only. In 1992, while serving as a policeman in Utravnic, he refused to join the Muslim forces and, as a consequence, he was accused of having stolen 40 rifles from the police station. Later, he was placed in a special unit from which he could not resign. Resenting police attacks on Muslims and being unable to handle the mental and physical stress, he later deserted from that unit. Further, in June 1993 Kraljeva Sutjeska was occupied by Muslim troops. Along with other residents, the applicant participated in the defence of the village. Later, he was forced to leave the village and, on 15 March 1994, he arrived in Croatia with his parents and siblings. Allegedly, if returned to Croatia, he would risk being sent to Bosnia-Hercegovina where he could be sentenced to 20 years in prison for refusing to bear arms.", "On 30 September 1994 the National Immigration Board (Statens invandrarverk) rejected the applicant\u2019s request and ordered his deportation to Croatia. The Board called into question the credibility of the applicant\u2019s statements concerning his military and police activities as he had changed the statements during the course of the investigation. Finding that the applicant held both Bosnian and Croatian citizenship, the Board referred to a decision concerning asylum seekers with such double nationality taken by the Swedish Government on 26 May 1994. According to the decision, the prevailing situation in Bosnia-Hercegovina rendered deportations to that country impossible. Consequently, in assessing applications from these asylum seekers, the crucial question was whether they could be afforded protection in Croatia. The Board noted that, according to available information, persons in the applicant\u2019s situation did not risk to be sent from Croatia to Bosnia-Hercegovina against their will. Moreover, no acts of warfare had taken place on Croatian territory for some time and a cease-fire had been agreed upon by the contending parties. Thus, there was no apparent risk that Croatian citizens in general would be forced to take part in armed conflict. Further, the applicant had failed to show that he personally faced such a risk.", "The applicant appealed to the Aliens Appeals Board (Utl\u00e4nningsn\u00e4mnden). On 29 March 1995 the Appeals Board, sharing the opinion of the Immigration Board, rejected the appeal.", "The applicant later requested a temporary residence permit in Sweden. By a decision of 20 June 1995 the Appeals Board granted the applicant such a permit until 1 December 1995 and quashed the deportation order. In so doing, the Board referred to a Government decision of 5 May 1995 according to which the worsened situation in Croatia constituted an impediment to the deportation of asylum seekers to that country.", "On 20 November 1995 the applicant submitted a new application for a residence permit. He referred to his previous statements. Further, in a final statement of 23 March 1997 he claimed that he should be granted a residence permit on humanitarian grounds on account of the time he had spent in Sweden.", "On 27 March 1997 the Immigration Board rejected the applicant\u2019s new application and ordered his deportation to Croatia. The Board had regard to a guiding decision taken by the Swedish Government on 28 November 1996. In that decision the Government stated, inter alia, the following:", "(Translation)", "\u201cAs regards the situation in Croatia and the region, the ongoing peace process, based on a general agreement on peace in Bosnia-Hercegovina, has lead to stabilisation. The relations between Croatia and the Federal Republic of Yugoslavia have improved. Croatia has been admitted into the Council of Europe. Acts of warfare have not occurred on Croatian territory since August 1995. The risk of further conflicts appears unlikely. The general situation in Croatia has improved in such a way since the Government\u2019s previous assessment in May 1995 that nowadays Croatian citizens can generally be afforded protection in Croatia.\u201d", "The Immigration Board noted that the Swedish Government\u2019s view had been confirmed by the United Nations High Commissioner for Refugees (UNHCR) and the Croatian Government. As in the previous decisions taken concerning the applicant, the Board found that he could not be sent back to Bosnia-Hercegovina. With regard to the possible deportation to Croatia, the Board noted that no new circumstances had been invoked by the applicant. Consequently, the Immigration Board shared the opinions expressed in its decision of 30 September 1994 and the decision taken by the Aliens Appeals Board on 29 March 1995. Furthermore, the Immigration Board found that there were no reasons of a humanitarian nature to grant the applicant a residence permit.", "Following the applicant\u2019s appeal, the Appeals Board, on 2 June 1997, quashed the Immigration Board\u2019s decision and referred the case back for re-examination. The Appeals Board found that no oral hearing had been held by the Immigration Board, as required by law.", "The Immigration Board held an oral hearing in the case on 11 June 1997 during which the applicant stated that, despite the fact that he held a Croatian passport, he did not possess the full rights of a Croatian citizen. Moreover, his parents and one of his brothers lived in Croatia where they had difficulties to support themselves. In Croatia nothing in particular would happen to the applicant but it would be hard for him to find work and a place to live.", "In a decision of 30 June 1997 the Immigration Board rejected the applicant\u2019s new application on the same grounds as those expressed in the quashed decision. On 17 April 1998 the Appeals Board rejected the applicant\u2019s appeal. The Appeals Board noted that the applicant is an ethnic Croat and, as such, would be at risk in his Bosnian home district, dominated by Muslims. However, the applicant\u2019s statements did not indicate that he would face ill-treatment in Croatia.", "In medical certificates issued by the psychiatric clinic at the hospital in Karlskrona on 30 November 1998 and 19 January 1999 chief physician G.S. and therapist I.S. stated that the applicant had undergone treatment for some time at the clinic for a post-traumatic stress syndrome. Allegedly, the enforcement of the deportation order would seriously impair his mental state and could lead to a suicide attempt. The clinic submitted the first certificate to the Aliens Appeals Board and requested that the enforcement be suspended.", "By a decision of 9 December 1998 the Appeals Board suspended the enforcement of the deportation order for an unspecified period of time."
str2="The applicant , born in DATE , presently resides in GPE , GPE . In his application form , he states that he has NORP nationality .", "", "On DATE the applicant arrived in GPE and requested asylum . He stated that he was NORP . Before leaving for GPE , he had been living in PERSON . He held also a NORP passport but this was to be considered as a travel document only . In DATE , while serving as a policeman in GPE , he refused to join the NORP forces and , as a consequence , he was accused of having stolen CARDINAL rifles from the police station . Later , he was placed in a special unit from which he could not resign . Resenting police attacks on NORP and being unable to handle the mental and physical stress , he later deserted from that unit . Further , in DATE PERSON was occupied by NORP troops . Along with other residents , the applicant participated in the defence of the village . Later , he was forced to leave the village and , on DATE , he arrived in GPE with his parents and siblings . Allegedly , if returned to GPE , he would risk being sent to GPE where he could be sentenced to DATE in prison for refusing to bear arms .", "On DATE ORG ( ORG invandrarverk ) rejected the applicant \u2019s request and ordered his deportation to GPE . ORG called into question the credibility of the applicant \u2019s statements concerning his military and police activities as he had changed the statements during the course of the investigation . Finding that the applicant held both NORP and NORP citizenship , the ORG referred to a decision concerning asylum seekers with such double nationality taken by ORG on DATE . According to the decision , the prevailing situation in GPE rendered deportations to that country impossible . Consequently , in assessing applications from these asylum seekers , the crucial question was whether they could be afforded protection in GPE . ORG noted that , according to available information , persons in the applicant \u2019s situation did not risk to be sent from GPE to GPE against their will . Moreover , no acts of warfare had taken place on NORP territory for some time and a cease - fire had been agreed upon by the contending parties . Thus , there was no apparent risk that NORP citizens in general would be forced to take part in armed conflict . Further , the applicant had failed to show that he personally faced such a risk .", "The applicant appealed to ORG ( Utl\u00e4nningsn\u00e4mnden ) . On DATE ORG , sharing the opinion of ORG , rejected the appeal .", "The applicant later requested a temporary residence permit in GPE . By a decision of DATE ORG granted the applicant such a permit until DATE and quashed the deportation order . In so doing , ORG referred to a Government decision of CARDINAL DATE according to which the worsened situation in GPE constituted an impediment to the deportation of asylum seekers to that country .", "On DATE the applicant submitted a new application for a residence permit . He referred to his previous statements . Further , in a final statement of DATE he claimed that he should be granted a residence permit on humanitarian grounds on account of the time he had spent in GPE .", "On DATE ORG rejected the applicant \u2019s new application and ordered his deportation to GPE . ORG had regard to a guiding decision taken by ORG on DATE . In that decision the ORG stated , inter alia , the following :", "( Translation )", "\u201c As regards the situation in GPE and the region , the ongoing peace process , based on a general agreement on peace in GPE , has lead to stabilisation . The relations between GPE and GPE have improved . GPE has been admitted into ORG . Acts of warfare have not occurred on NORP territory since DATE . The risk of further conflicts appears unlikely . The general situation in GPE has improved in such a way since the Government \u2019s previous assessment in DATE that nowadays NORP citizens can generally be afforded protection in GPE . \u201d", "ORG noted that ORG view had been confirmed by the ORG High Commissioner for Refugees ( ORG ) and ORG . As in the previous decisions taken concerning the applicant , ORG found that he could not be sent back to GPE . With regard to the possible deportation to GPE , the ORG noted that no new circumstances had been invoked by the applicant . Consequently , ORG shared the opinions expressed in its decision of DATE and the decision taken by ORG on DATE . Furthermore , ORG found that there were no reasons of a humanitarian nature to grant the applicant a residence permit .", "Following the applicant \u2019s appeal , ORG , on DATE , quashed ORG decision and referred the case back for re - examination . ORG found that no oral hearing had been held by ORG , as required by law .", "ORG held an oral hearing in the case on DATE during which the applicant stated that , despite the fact that he held a NORP passport , he did not possess the full rights of a NORP citizen . Moreover , his parents and one of his brothers lived in GPE where they had difficulties to support themselves . In GPE nothing in particular would happen to the applicant but it would be hard for him to find work and a place to live .", "In a decision of DATE ORG rejected the applicant \u2019s new application on the same grounds as those expressed in the quashed decision . On DATE ORG rejected the applicant \u2019s appeal . ORG noted that the applicant is an ethnic NORP and , as such , would be at risk in his NORP home district , dominated by NORP . However , the applicant \u2019s statements did not indicate that he would face ill - treatment in GPE .", "In medical certificates issued by the psychiatric clinic at the hospital in GPE on DATE and DATE chief physician PERSON and therapist PERSON stated that the applicant had undergone treatment for some time at the clinic for a post - traumatic stress syndrome . Allegedly , the enforcement of the deportation order would seriously impair his mental state and could lead to a suicide attempt . The clinic submitted the first certificate to ORG and requested that the enforcement be suspended .", "By a decision of DATE ORG suspended the enforcement of the deportation order for an unspecified period of time ."

differences = find_differences(str1, str2)

# 打印结果
print("不一样的子串: ", differences)

不一样的子串:  {'str1_diff': [('The applicant, born in 1968, presently resides in Karlskrona, Sweden. In his application form, he states that he has Croatian nationality.',), ('On 23 March 1994 the applicant arrived in Sweden and requested asylum. He stated that he was a Bosnian Catholic with Bosnian citizenship. Before leaving for Sweden, he had been living in Kraljeva Sutjeska. He held also a Croatian passport but this was to be considered as a travel document only. In 1992, while serving as a policeman in Utravnic, he refused to join the Muslim forces and, as a consequence, he was accused of having stolen 40 rifles from the police station. Later, he was placed in a special unit from which he could not resign. Resenting police attacks on Muslims and being unable to handle the mental and physical stress, he later deserted from that unit. Further, in June 1993 Kraljeva Sutjeska was occupied by Muslim troops. Along with other residents, the applicant participated in the defence of the village